In [1]:
import joblib
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
features = pd.read_csv(r"data\numerical\distance_train.csv")
targets = pd.read_csv(r"data\numerical\train_target.csv")

In [4]:
features_test = pd.read_csv(r"data\numerical\test_distance_features.csv")
targets_test = pd.read_csv(r"data\numerical\test_target.csv")

In [5]:
scaler = MinMaxScaler().fit(features.values)

In [6]:
features = scaler.transform(features.values)
features_test = scaler.transform(features_test.values)

In [ ]:
joblib.dump(scaler, "scaler.save")

In [8]:
import tensorflow.keras as keras
checkpoint = keras.callbacks.ModelCheckpoint("best_model.h5", monitor="val_accuracy", verbose=2, save_best_only=True, save_weights_only=False, mode= "auto", save_freq = "epoch")

In [9]:
model = Sequential()
model.add(Dense(1035, input_dim=1035))
model.add(LeakyReLU(0.8))
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(LeakyReLU(0.8))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(LeakyReLU(0.8))
model.add(Dropout(0.5))
model.add(Dense(27, activation="softmax"))

In [10]:
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adamax(learning_rate=1e-4), metrics=["accuracy", keras.metrics.Recall(), keras.metrics.Precision()])

In [ ]:
model.fit(features, targets, epochs=15000, batch_size=512, verbose=2, callbacks=checkpoint, validation_data=(features_test, targets_test))